In [ ]:
import matplotlib.pyplot as plt
import os
from glob import glob as glob
import cv2
import random
from pylab import *

%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D
from keras.layers import BatchNormalization , Activation , MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau
from IPython.display import SVG , Image
from tensorflow.keras.utils import plot_model
import tensorflow as tf
print("TensorFlow Version" , tf.__version__)

In [ ]:
images = glob("/kaggle/input/fer2013/train/**/**")
for i in range(9):
    image = choice(images)
    plt.figure(figsize=(15,15) )
    plt.subplot(331+i)
    plt.imshow(cv2.imread(image));plt.axis('off')

In [ ]:
img_size = 48
batch_size = 64
datagen_train = ImageDataGenerator(horizontal_flip = True)
train_generator = datagen_train.flow_from_directory("/kaggle/input/fer2013/train", target_size = (img_size, img_size) , color_mode = "grayscale" , batch_size = batch_size , class_mode = "categorical" , shuffle = True )

datagen_validation = ImageDataGenerator(horizontal_flip = True)
validation_generator = datagen_train.flow_from_directory("/kaggle/input/fer2013/test", target_size = (img_size, img_size) , color_mode = "grayscale" , batch_size = batch_size , class_mode = "categorical" , shuffle = True )


In [ ]:
def Convolution(input_tensor , filters , kernel_size):
    x = Conv2D(filters = filters , kernel_size = kernel_size , padding = 'same')(input_tensor)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size= (2,2))(x)
    x = Dropout(0.25)(x)
    return x

In [ ]:
def Dense_f(input_tensor , nodes):
    x = Dense(nodes)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)
    return x
    

In [ ]:
def model_fer(input_shape):
    inputs = Input(input_shape)
    conv1 = Convolution(inputs, 64 , (3,3))
    conv2 = Convolution(inputs, 128 , (5,5))
    conv3 = Convolution(inputs, 512 , (3,3))
    
    conv4 = Convolution(inputs, 512 , (3,3))
    
    flatten = Flatten()(conv4)
    dense_1 = Dense_f(flatten , 256)
    dense_2 = Dense_f(flatten , 512)
    output = Dense(7 , activation = "softmax")(dense_2)
    model = Model(inputs = [inputs] , outputs = [output])
    
    opt = Adam(lr = 0.0005)
    
    model.compile(loss = ['categorical_crossentropy'] , optimizer = opt , metrics = ['accuracy'])
    return model

In [ ]:
model = model_fer((48,48,1))
model.summary()

In [ ]:
epochs = 15
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

checkpoint = ModelCheckpoint("model_weights.h5" , monitor = 'val_accuracy' , save_weights_only = True , mode = 'max' , verbose = 1 )

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss' , factor = 0.1 , patience = 2 , min_lr = 0.00001)

callbacks = [checkpoint, reduce_lr]

In [ ]:
history = model.fit(
x= train_generator,
steps_per_epoch = steps_per_epoch , 
epochs = epochs ,
validation_data = validation_generator , 
validation_steps = validation_steps , 
callbacks = callbacks
)

In [ ]:
model.evaluate(validation_generator)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.legend(['train' , 'validation'] , loc = 'upper left' )
plt.subplots_adjust(top = 1.0, bottom = 0.0 , right = 0.95 , left = 0.0 , hspace = 0.25 , wspace = 0.35)

In [ ]:
model_json = model.to_json()
with open('model_a1.json' , 'w') as json_file:
    json_file.write(model_json)